In [ ]:
!pip install unidecode
!pip install contractions
!pip install demoji
!pip install num2words
!pip install SpellChecker
!pip install pandas-profiling[notebook]
!pip install ydata_profiling

  Using cached visions-0.7.4-py3-none-any.whl.metadata (5.9 kB)
Using cached visions-0.7.4-py3-none-any.whl (102 kB)
  Attempting uninstall: visions
    Found existing installation: visions 0.7.6
    Uninstalling visions-0.7.6:
      Successfully uninstalled visions-0.7.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.12.2 requires visions[type_image_path]<0.8.0,>=0.7.5, but you have visions 0.7.4 which is incompatible.
  Using cached visions-0.7.6-py3-none-any.whl.metadata (11 kB)
Using cached visions-0.7.6-py3-none-any.whl (104 kB)
  Attempting uninstall: visions
    Found existing installation: visions 0.7.4
    Uninstalling visions-0.7.4:
      Successfully uninstalled visions-0.7.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
import re
import nltk
import spacy
import string
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import demoji
import contractions
import unidecode
from num2words import num2words
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

In [ ]:
!wget https://gitlab.com/gguibon/ensae-ml4nlp-storage/-/raw/master/bbc-news-data.zip
!unzip bbc-news-data.zip

--2025-02-10 07:48:30--  https://gitlab.com/gguibon/ensae-ml4nlp-storage/-/raw/master/bbc-news-data.zip
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1896286 (1.8M) [application/octet-stream]
Saving to: ‘bbc-news-data.zip.1’

bbc-news-data.zip.1 100%[===================>]   1.81M  --.-KB/s    in 0.03s   

2025-02-10 07:48:31 (56.8 MB/s) - ‘bbc-news-data.zip.1’ saved [1896286/1896286]

Archive:  bbc-news-data.zip
replace bbc-news-data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: bbc-news-data.csv       


In [ ]:
import pandas as pd
df_bbc = pd.read_csv('bbc-news-data.csv', sep='\t', on_bad_lines='skip')
df_bbc.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [ ]:
print(df_bbc['category'].unique())

['business' 'entertainment' 'politics' 'sport' 'tech']


In [ ]:
profile = ProfileReport(df_bbc, title="BBC News Dataset Profiling Report")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

2. Ambil data dari Twitter dengan keyword: "**Arsenal**", maksimal 200 data. Setelah itu tampilkan dalam dataframe. Ambil hanya kolom **full_text** untuk dianalisa. Lakukan ***cleaning*** (tidak boleh ada simbol), ***case_folding*** (huruf diubah menjadi kecil semua), ***tokenisasi*** (dipecah menjadi token), ***filtering*** dan ***stemming*** (diubah menjadi kata dasar), untuk kolom **full_text**.

In [ ]:
pd.options.display.max_columns=None
pd.options.display.max_rows=None
pd.options.display.max_colwidth=None

nltk.download('twitter_samples')

for name in twitter_samples.fileids():
    print(f' - {name}')

 - negative_tweets.json
 - positive_tweets.json
 - tweets.20150430-223406.json


[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [ ]:
negative_tweets = twitter_samples.strings("negative_tweets.json")
df_neg = pd.DataFrame(negative_tweets, columns=['text'])
df_neg['label'] = 0

positive_tweets = twitter_samples.strings("positive_tweets.json")
df_pos = pd.DataFrame(positive_tweets, columns=['text'])
df_pos['label'] = 1

df = pd.concat([df_pos, df_neg])
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
print(f'Shape of the whole data is: {df.shape[0]} rows and {df.shape[1]} columns')

Shape of the whole data is: 10000 rows and 2 columns


In [ ]:
#take some tweets with arsenal world
arsenal_tweets = df[df['text'].str.contains("Arsenal", na=False, case=False)]
arsenal_tweets.head()

,text,label
623,"@arsenalnewsasit :) Thanks, wishing you back also. #LazyWeekend",1
5816,"@Mburu__ Inter 3 UCL, Arsenal... Small team, Right! :)",1
6766,"#FF @EmiratesInsight for the best, most recent Arsenal news on your TL. We were hacked and it's a new account. Give it a follow lads :)",1
7337,@arsenalnewsasit You're welcome Neil :). What's the next match in Emirates?,1
9284,"@arsenalnewsasit Thanks again, your time 16.20 right? That's perfect timing for me here :).",1


In [ ]:
arsenal_tweets = df['text']
print(arsenal_tweets.head())

0                                               @normabattle thx for the feedback. Glad you gained value from it. :)
1    @Awaishooo Shahid Afridi tou young hi hai na abhi. talk about Mahnor baloch. she will never going to be old. :)
2                                                       i see ur bio @xannindy 😆😆 u don't break my heart actually :(
3                                                                                           @xjisoobelle no money :(
4                                All ice cream vans are busy :( Maybe later...  #UberIceCream http://t.co/t6GAoKfAp1
Name: text, dtype: object


In [ ]:
#Cleaning
import re

def clean_text(text):
  text = re.sub(r'[^\w\s]', '', text)
  return text

text_data_cleaned = arsenal_tweets.apply(clean_text)
print(text_data_cleaned.head())

0                                              normabattle thx for the feedback Glad you gained value from it 
1    Awaishooo Shahid Afridi tou young hi hai na abhi talk about Mahnor baloch she will never going to be old 
2                                                       i see ur bio xannindy  u dont break my heart actually 
3                                                                                        xjisoobelle no money 
4                                     All ice cream vans are busy  Maybe later  UberIceCream httptcot6GAoKfAp1
Name: text, dtype: object


In [ ]:
#Lowercase
def case_folding(text):
  text = text.lower()
  return text

text_data_folded = text_data_cleaned.apply(case_folding)
print(text_data_folded.head())


0                                              normabattle thx for the feedback glad you gained value from it 
1    awaishooo shahid afridi tou young hi hai na abhi talk about mahnor baloch she will never going to be old 
2                                                       i see ur bio xannindy  u dont break my heart actually 
3                                                                                        xjisoobelle no money 
4                                     all ice cream vans are busy  maybe later  ubericecream httptcot6gaokfap1
Name: text, dtype: object


In [ ]:
#Tokenization
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

def tokenize_text(text):
  return word_tokenize(text)

text_data_tokenized = text_data_folded.apply(tokenize_text)
print(text_data_tokenized.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


0                                                       [normabattle, thx, for, the, feedback, glad, you, gained, value, from, it]
1    [awaishooo, shahid, afridi, tou, young, hi, hai, na, abhi, talk, about, mahnor, baloch, she, will, never, going, to, be, old]
2                                                                 [i, see, ur, bio, xannindy, u, dont, break, my, heart, actually]
3                                                                                                         [xjisoobelle, no, money]
4                                                [all, ice, cream, vans, are, busy, maybe, later, ubericecream, httptcot6gaokfap1]
Name: text, dtype: object


In [ ]:
#Stemming dan Filterisasi
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


def filtering_text(text):
    filtered_sentence = [w for w in text if not w in stop_words]
    return filtered_sentence

text_data_filtered = text_data_tokenized.apply(filtering_text)
print(text_data_filtered.head())


def stemming_text(text):
    stemmed_sentence = [stemmer.stem(w) for w in text]
    return stemmed_sentence


text_data_stemmed = text_data_filtered.apply(stemming_text)
print(text_data_stemmed.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0                                                      [normabattle, thx, feedback, glad, gained, value]
1    [awaishooo, shahid, afridi, tou, young, hi, hai, na, abhi, talk, mahnor, baloch, never, going, old]
2                                              [see, ur, bio, xannindy, u, dont, break, heart, actually]
3                                                                                   [xjisoobelle, money]
4                                [ice, cream, vans, busy, maybe, later, ubericecream, httptcot6gaokfap1]
Name: text, dtype: object
0                                                       [normabattl, thx, feedback, glad, gain, valu]
1    [awaishooo, shahid, afridi, tou, young, hi, hai, na, abhi, talk, mahnor, baloch, never, go, old]
2                                             [see, ur, bio, xannindi, u, dont, break, heart, actual]
3                                                                                  [xjisoobel, money]
4                               [ice, cre

Perform the Term Weighting - TF-IDF calculation for the following set of documents:

- d1 = "aku cinta kamu" (I love you)
- d2 = "aku cinta kamu dan dia" (I love you and her/him)
- d3 = "aku cinta kamu tapi kamu cinta dia" (I love you but you love her/him)
- d4 = "aku memang cinta kamu tapi aku milih dia" (I do love you but I choose her/him)

In [ ]:
d1 = "aku cinta kamu"
d2 = "aku cinta kamu dan dia"
d3 = "aku cinta kamu tapi kamu cinta dia"
d4 = "aku memang cinta kamu tapi aku milih dia"

d_string = f"{d1}{d2}{d3}{d4}"
d_semua = d1.split()+d2.split()+d3.split()+d4.split()
d_list = [d1, d2, d3, d4]

print(d_list)

['aku cinta kamu', 'aku cinta kamu dan dia', 'aku cinta kamu tapi kamu cinta dia', 'aku memang cinta kamu tapi aku milih dia']


In [ ]:
kamus_kata = {}
for data in d_semua:
  kamus_kata[data] = d_string.count(data)

print(kamus_kata)

{'aku': 5, 'cinta': 5, 'kamu': 5, 'dan': 1, 'dia': 3, 'tapi': 2, 'memang': 1, 'milih': 1}


In [ ]:
data_dict = {}
data_list = []
for kalimat in d_list:
  for kata in d_semua:
    data_dict[kata] = kalimat.count(kata)/len(kalimat.split())
  data_list.append(data_dict)
  data_dict = {}

print(data_list)

[{'aku': 0.3333333333333333, 'cinta': 0.3333333333333333, 'kamu': 0.3333333333333333, 'dan': 0.0, 'dia': 0.0, 'tapi': 0.0, 'memang': 0.0, 'milih': 0.0}, {'aku': 0.2, 'cinta': 0.2, 'kamu': 0.2, 'dan': 0.2, 'dia': 0.2, 'tapi': 0.0, 'memang': 0.0, 'milih': 0.0}, {'aku': 0.14285714285714285, 'cinta': 0.2857142857142857, 'kamu': 0.2857142857142857, 'dan': 0.0, 'dia': 0.14285714285714285, 'tapi': 0.14285714285714285, 'memang': 0.0, 'milih': 0.0}, {'aku': 0.25, 'cinta': 0.125, 'kamu': 0.125, 'dan': 0.0, 'dia': 0.125, 'tapi': 0.125, 'memang': 0.125, 'milih': 0.125}]


In [ ]:
print("Term Frequency")
for i in range(len(data_list)):
  print(f"Data{i+1} : {data_list[i]}")

cek_ulang = []
for d in d_list:
  periksa = set(d.split())
  cek_ulang += list(periksa)
for cek in cek_ulang:
  kamus_kata[cek] = cek_ulang.count(cek)


Term Frequency
Data1 : {'aku': 0.3333333333333333, 'cinta': 0.3333333333333333, 'kamu': 0.3333333333333333, 'dan': 0.0, 'dia': 0.0, 'tapi': 0.0, 'memang': 0.0, 'milih': 0.0}
Data2 : {'aku': 0.2, 'cinta': 0.2, 'kamu': 0.2, 'dan': 0.2, 'dia': 0.2, 'tapi': 0.0, 'memang': 0.0, 'milih': 0.0}
Data3 : {'aku': 0.14285714285714285, 'cinta': 0.2857142857142857, 'kamu': 0.2857142857142857, 'dan': 0.0, 'dia': 0.14285714285714285, 'tapi': 0.14285714285714285, 'memang': 0.0, 'milih': 0.0}
Data4 : {'aku': 0.25, 'cinta': 0.125, 'kamu': 0.125, 'dan': 0.0, 'dia': 0.125, 'tapi': 0.125, 'memang': 0.125, 'milih': 0.125}


In [ ]:
import math
data_idf = {}
for kata in kamus_kata:
  hitung = math.log(len(d_list)/kamus_kata[kata], 10)
  data_idf[kata] = hitung

print(data_idf)

{'aku': 0.0, 'cinta': 0.0, 'kamu': 0.0, 'dan': 0.6020599913279623, 'dia': 0.1249387366082999, 'tapi': 0.30102999566398114, 'memang': 0.6020599913279623, 'milih': 0.6020599913279623}


In [ ]:
# TF-IDF
hasil = []
data_sementara = {}

for index in range (len(data_list)):
  for i in data_list[index]:
    hitung = data_list[index][i] * data_idf[i]
    data_sementara[i] = round(hitung, 3)
  hasil.append(data_sementara)
  data_sementara = {}

for i in range(len(hasil)):
  print(f"Data {i} : {hasil[i]}")

Data 0 : {'aku': 0.0, 'cinta': 0.0, 'kamu': 0.0, 'dan': 0.0, 'dia': 0.0, 'tapi': 0.0, 'memang': 0.0, 'milih': 0.0}
Data 1 : {'aku': 0.0, 'cinta': 0.0, 'kamu': 0.0, 'dan': 0.12, 'dia': 0.025, 'tapi': 0.0, 'memang': 0.0, 'milih': 0.0}
Data 2 : {'aku': 0.0, 'cinta': 0.0, 'kamu': 0.0, 'dan': 0.0, 'dia': 0.018, 'tapi': 0.043, 'memang': 0.0, 'milih': 0.0}
Data 3 : {'aku': 0.0, 'cinta': 0.0, 'kamu': 0.0, 'dan': 0.0, 'dia': 0.016, 'tapi': 0.038, 'memang': 0.075, 'milih': 0.075}
